In [1]:
import pandas as pd
import nltk

In [3]:
# Load data frame
#df = pd.read_csv('/media/zainkhan/USB30FD/suspended-trump-tweets.txt', encoding='latin1', error_bad_lines=False, warn_bad_lines=False, header=None)
df = pd.read_csv('/media/zainkhan/My Passport/15Days/22-7Clinton.txt', sep="\n", encoding='latin1', error_bad_lines=False, warn_bad_lines=False, header=None)
df = df[0]
df.head()

0    RT @Always_Trump: BOMBSHELL:  Clinton Campaign...
1    RT @c5hardtop1999: Hillary's foreign policy ha...
2    If she wins, be aware that the one who warms y...
3    RT @wikileaks: Bernie Sanders wife Jane Sander...
4    RT @wikileaks: Bernie Sanders wife Jane Sander...
Name: 0, dtype: object

In [4]:
# Remove stop words
df = df.str.lower().str.split()
df.head()

0    [rt, @always_trump:, bombshell:, clinton, camp...
1    [rt, @c5hardtop1999:, hillary's, foreign, poli...
2    [if, she, wins,, be, aware, that, the, one, wh...
3    [rt, @wikileaks:, bernie, sanders, wife, jane,...
4    [rt, @wikileaks:, bernie, sanders, wife, jane,...
Name: 0, dtype: object

In [5]:
stop = nltk.corpus.stopwords.words('english')
df = df.dropna().apply(lambda x: [item for item in x if item not in stop])
df.head()

0    [rt, @always_trump:, bombshell:, clinton, camp...
1    [rt, @c5hardtop1999:, hillary's, foreign, poli...
2    [wins,, aware, one, warms, bed, may, involunta...
3    [rt, @wikileaks:, bernie, sanders, wife, jane,...
4    [rt, @wikileaks:, bernie, sanders, wife, jane,...
Name: 0, dtype: object

In [6]:
# Separate into sentences
sentences = df.tolist()
print(sentences[:2])

[['rt', '@always_trump:', 'bombshell:', 'clinton', 'campaign', 'admits', 'globalization', 'hurts', 'wages', '&amp;', 'jobs!', 'want', 'it,', 'hillary?', '#podestaemails'], ['rt', '@c5hardtop1999:', "hillary's", 'foreign', 'policy', 'consisted', 'selling', 'access', 'clinton', 'foundation', '&amp;', 'promoting', 'muslim', 'brotherho']]


In [7]:
# Clean and remove illegal characters
import re
def clean(s):
    for i, w in enumerate(s):
        if contains_illegal(w):
            s[i] = ''
        else:
            if 'cli' in w:
                s[i] = 'clinton'
            elif 'hrc' in w or 'hil' in w:
                s[i] = 'hillary'
            elif 'tru' in w or 'trmp' in w:
                s[i] = 'trump'
            elif 'bern' in w:
                s[i] = 'bernie'
            elif 'sander' in w:
                s[i] = 'sanders'
            extras = '0123456789~_=*^,.-`%""+#&"!?<>:;/\\\'()[]{}$|\x91\x92\x93\x94\x96\x97'
            s[i] = s[i].translate({ord(c):'' for c in extras})
            if len(s[i]) < 4:
                s[i] = ''
    return s
                    
def contains_illegal(w):
    illegal = ['@', '#', 'htt', 'via', 'desd']
    if any(x in w for x in illegal):
        return True
    if re.match("1", w) is not None:
        return True
    return False 

sentences = [clean(s) for s in sentences if len(s) > 0]
sentences = [list(filter(None, sentence)) for sentence in sentences]
print(sentences[:2])

[['bombshell', 'clinton', 'campaign', 'admits', 'globalization', 'hurts', 'wages', 'jobs', 'want', 'hillary'], ['hillary', 'foreign', 'policy', 'consisted', 'selling', 'access', 'clinton', 'foundation', 'promoting', 'muslim', 'brotherho']]


In [8]:
# Get adjective list to iterate over
adj = pd.read_csv('adjectives-character-richards.csv')
adj = adj.values.tolist()
adj = [word[0] for word in adj]
adj.append('corrupt')
adj.append('crooked')
adj[:5]

['accessible', 'active', 'adaptable', 'admirable', 'adventurous']

In [15]:
# Maps adjective occurences per sentence relative to a word
def adj_to_word(sentences, adj_list, target):
    adj_to_word = {}
    sentences_with_target = [s for s in sentences if target in s]

    for adj in adj_list:
        for sentence in sentences_with_target:
            if adj in sentence:
                if adj in adj_to_word:
                    adj_to_word[adj] += 1
                else:
                    adj_to_word[adj] = 1
    return adj_to_word

target_word = 'clinton'
d = adj_to_word(sentences, adj, target_word)

In [16]:
dlist = []
for key, value in d.items():
    temp = [key,value]
    dlist.append(temp)
dlist.sort(key=lambda x: x[1], reverse=True)
print(dlist[:3])

[['criminal', 98915], ['crooked', 73512], ['private', 61983]]


In [17]:
# min_count in W2V is 5
dlist = [s for s in dlist if s[1] > 5]

print(target_word)
for i in dlist:
    print(i[0], i[1])

clinton
criminal 98915
crooked 73512
private 61983
corrupt 44597
ignorant 34424
political 29481
open 19697
stupid 14368
sober 13002
honest 12161
false 11791
cold 11183
winning 11161
loyal 9857
confident 9542
alert 8676
strong 8092
liberal 7883
deep 7630
quiet 7618
clean 7187
friendly 7171
dirty 6964
transparent 6471
ordinary 6458
desperate 6190
responsible 5873
serious 5860
clever 5702
secure 5521
conservative 5493
firm 5486
directed 5273
grand 5237
brutal 4709
active 4700
intense 4457
selfish 4413
insulting 3869
tough 3854
scheming 3693
kind 3644
disturbing 3628
modern 3562
solitary 3540
simple 3426
dishonest 3374
solid 3337
ruined 3229
religious 3205
proud 3193
naive 3174
authoritarian 3106
crazy 3030
angry 3001
soft 2970
popular 2560
independent 2320
pompous 2312
weak 2263
busy 2173
questioning 2136
tolerant 1989
obvious 1958
progressive 1943
bizarre 1925
pure 1787
careless 1684
narrow 1650
intolerant 1614
brilliant 1611
demanding 1498
hateful 1476
sweet 1388
unhealthy 1378
suspicio

In [ ]:
# Word Embedding model
from gensim.models import Word2Vec

model = Word2Vec(sentences, size=100, window=5, min_count=3, workers=4)
model.train(sentences, total_examples=len(sentences), epochs=10)

In [ ]:
import bisect

def top_adjectives(model, token, topn=10):
    out = []
    word_to_sim = {}
    same = []
    for elem in dlist:
        if elem[0] == token:
            same = elem
    if same != []:
        unique_dlist = dlist[:]
        unique_dlist.remove(same)
    else:
        unique_dlist = dlist[:]
    for wo in unique_dlist:
        w = wo[0]
        word, similarity = w, model.wv.similarity(w1=token,w2=w)
        if len(out) < topn:
            bisect.insort(out, similarity)
            word_to_sim[similarity] = word 
        if similarity > out[0] and len(out) == topn and word not in word_to_sim.values():
            del out[0]
            bisect.insort(out, similarity)
            word_to_sim[similarity] = word
            
    adj_order = []
    for sim in out:
        adj_order.append([word_to_sim[sim], sim])
    adj_order.append([token, ''])
    return adj_order[::-1]

res = top_adjectives
for elem in top_adjectives(model, target_word):
    print(elem[0], elem[1])

In [ ]:
def network(model, token, topn=10):
    base = top_adjectives(model, token, topn)
    network = []
    network.append([base])
    for i in range(topn-1, 0, -1):
        new_related = top_adjectives(model, base[topn - i][0])
        network.append([new_related, [[0]], [[topn - i]]])

        for layer in range(topn-1, 0, -1):
            layered_related = top_adjectives(model, new_related[topn - layer][0])
            network.append([layered_related, [[topn - layer + 1]], [[topn - i]]])
    return network

net = network(model, target_word)
def print_network(net):
    for words in net:
        for items in words: 
            for word in items:
                if len(word) == 2:
                    print(word[0], word[1])
                else:
                    print(word[0])
        print('\n')

print_network(net)